Keeping only the drug: side effect entries where the drug is present in the list of drugs in drugbank_inhibitor_gsmm.csv file

In [2]:
import pandas as pd
df1 = pd.read_csv('C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/DrugBank/drugbank_inhibitors_gsmm.csv')
df2 = pd.read_csv('C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/SIDER/sider2_processed.csv')
df3 = pd.read_csv('C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/DrugBank/drugbank_inhibitors_gsmm_meta.csv')

filtered_data = df2[df2['dg_id'].isin(df1['dg_id'])]
filtered_data_meta = df2[df2['dg_id'].isin(df3['dg_id'])]
filtered_data.to_csv('C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/SIDER/sider2_processed.csv', index=False)
filtered_data_meta.to_csv('C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/SIDER/sider2_processed_meta.csv', index=False)



Generating reverse indexed list (side effect: causative drugs)

In [3]:
import pandas as pd

# Load CSV
df = pd.read_csv("C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/SIDER/sider2_processed.csv")
df1 = pd.read_csv("C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/SIDER/sider2_processed_meta.csv")

# Group by "Side Effect" and aggregate unique drugs
grouped_df = df.groupby("concept_id")["dg_id"].apply(lambda x: ", ".join(x.unique())).reset_index()
grouped_df1 = df1.groupby("concept_id")["dg_id"].apply(lambda x: ", ".join(x.unique())).reset_index()
# Filter for side effects with at least 10 causative drugs
filtered_df = grouped_df[grouped_df["dg_id"].str.split(", ").apply(len) >= 10]
filtered_df1 = grouped_df1[grouped_df1["dg_id"].str.split(", ").apply(len) >= 10]
# Save the filtered results
filtered_df.to_csv("C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/SIDER/side_effects_vs_causative_drugs.csv", index=False)
filtered_df1.to_csv("C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/SIDER/side_effects_vs_causative_drugs_meta.csv", index=False)

# Display the result
#print(filtered_df)


Finding no. of unique drugs left

In [8]:
import pandas as pd

# Load the filtered CSV file
filtered_data = pd.read_csv('C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/SIDER/side_effects_vs_causative_drugs.csv')
filtered_data_meta = pd.read_csv('C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/SIDER/side_effects_vs_causative_drugs_meta.csv')

# Split the 'drug' column (assuming column name is 'drug') and flatten the list
all_drugs = filtered_data['dg_id'].str.split(',').explode().str.strip()
all_drugs_meta = filtered_data_meta['dg_id'].str.split(',').explode().str.strip()

# Get the number of unique drugs
unique_drug_count = all_drugs.nunique()
unique_drug_count_meta = all_drugs_meta.nunique()

# Display the result
print(f"Number of unique drugs: {unique_drug_count}")
print(f"Number of unique drugs (only metabolic): {unique_drug_count_meta}")


Number of unique drugs: 137
Number of unique drugs (only metabolic): 97


Generate list of drug-gene target interaction mapping

In [11]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/DrugBank/drugbank_inhibitors_gsmm_meta.csv')
  # Replace with your actual file path
filtered_data = pd.read_csv('C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/SIDER/side_effects_vs_causative_drugs_meta.csv')

# Split the 'drug' column (assuming column name is 'drug') and flatten the list
all_drugs = filtered_data['dg_id'].str.split(',').explode().str.strip()

# Get the number of unique drugs
unique_drugs = all_drugs.unique()

# Fill NaN values with an empty string (or drop them)
df["target_gene_name"] = df["target_gene_name"].fillna("")  # Alternative: df.dropna(subset=["gene"], inplace=True)

# Group by 'drug' and aggregate genes into a single column
grouped_df = df.groupby("dg_id", as_index=False).agg({"target_gene_name": lambda x: ", ".join(x.dropna().astype(str).unique())})

final_df=grouped_df[grouped_df['dg_id'].isin(unique_drugs)]
# Save the processed data
final_df.to_csv("C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/DrugBank/drug_gene_targets_meta.csv", index=False)

# Display the first few rows
#print(grouped_df.head())


To convert the descriptive gene names back to model gene names

In [13]:
import pandas as pd

# Load the first CSV (drug-gene mappings)
drugs_df = pd.read_csv("C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/DrugBank/drug_gene_targets_meta.csv")  # Columns: Drug, Gene_Target_Convention1

# Load the second CSV (gene name conversions)
gene_conversion_df = pd.read_csv("C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/GSMMs/gene_descriptions.csv")  # Columns: Convention1, Convention2

# Explode drug-gene targets into separate rows
drugs_df["target_gene_name"] = drugs_df["target_gene_name"].str.split(", ")
drugs_df = drugs_df.explode("target_gene_name")

# Merge to get convention2 names
merged_df = drugs_df.merge(gene_conversion_df, left_on="target_gene_name", right_on="genes_desc", how="left")

# Replace convention1 names with convention2
merged_df["Gene_Target_Convention2"] = merged_df["gene_ids"].fillna(merged_df["target_gene_name"])

# Drop unnecessary columns
merged_df = merged_df.drop(columns=["target_gene_name", "genes_desc", "gene_ids"])

# Group back by drug, joining multiple targets with commas
final_df = merged_df.groupby("dg_id")["Gene_Target_Convention2"].apply(lambda x: ", ".join(x)).reset_index()

# Rename column
final_df = final_df.rename(columns={"Gene_Target_Convention2": "target_gene_name"})

# Save to new CSV
final_df.to_csv("C:/Users/Saathvik/Documents/MATLAB/Project_Liver/data/DrugBank/drug_gene_targets_meta_1.csv", index=False)



# Redundant 

In [6]:
import pandas as pd

# Load CSV file 'a' with a header
a = pd.read_csv("C:/Users/Saathvik/Documents/MATLAB/Project_Recon3D/data/DrugBank/drugbank_inhibitors_gsmm.csv")

# Load CSV file 'b' without a header
b = pd.read_excel("C:/Users/Saathvik/Downloads/Mammalian_Metabolic_Final.xlsx")
a['gene_normalized'] = a['target_gene_name'].str.lower()
b['Gene Symbol'] = b['Gene Symbol'].str.lower()
# Filter records in 'a' where the 'gene' column exists in the 5th column of 'b'
filtered_a = a[a['gene_normalized'].isin(b['Gene Symbol'])]
filtered_a = filtered_a.drop(columns=['gene_normalized'])

# Save the filtered records to a new CSV file
filtered_a.to_csv('filtered_a.csv', index=False)

print("Filtered records saved to 'filtered_a.csv'")

Filtered records saved to 'filtered_a.csv'


In [ ]:

a['gene_normalized'] = a['target_gene_name'].str.lower()
b[3] = b[3].str.lower()
# Filter records in 'a' where the 'gene' column exists in the 5th column of 'b'
filtered_a = a[a['gene_normalized'].isin(b[3])]
filtered_a = filtered_a.drop(columns=['gene_normalized'])

# Save the filtered records to a new CSV file
filtered_a.to_csv('filtered_a.csv', index=False)

print("Filtered records saved to 'filtered_a.csv'")

In [14]:

a['gene_normalized'] = a['target_gene_name'].str.lower()
b[3] = b[3].str.lower()
# Filter records in 'a' where the 'gene' column exists in the 5th column of 'b'
filtered_a = a[a['gene_normalized'].isin(b[3])]
filtered_a = filtered_a.drop(columns=['gene_normalized'])

# Save the filtered records to a new CSV file
filtered_a.to_csv('filtered_a.csv', index=False)

print("Filtered records saved to 'filtered_a.csv'")


NameError: name 'a' is not defined